In [9]:
%load_ext autoreload
%autoreload 2

In [17]:
import sys
sys.path.append('../src')

import itertools
import random
import json
import numpy as np
from collections import Counter

In [18]:
import util
from uscode import USCode
from citation_network import CitationNetwork
from cluster import Clustering

In [3]:
usc = USCode.from_json('../data/usc201909.json')
print(len(usc.titles()))

53


In [24]:
# import networkx as nx

# class TitleCitationNetwork(nx.DiGraph):
#     def __init__(self, uscode):
#         super().__init__()

#         for sec in uscode.iter_sections():
#             title_id = util.extract_title_id(sec.id)
#             self.add_node(title_id)
#             for ref_id, ref_count in sec.refs.items():
#                 ref_title_id = util.extract_title_id(ref_id)
#                 self.update_edge(title_id, ref_title_id, weight=ref_count)

#     def sinks_from(self, node):
#         citations = nx.descendants(self, node)
#         return {c for c in citations if self.out_degree(c)}
    
#     def update_edge(self, src, dst, weight):
#         if self.has_edge(src, dst):
#             weight += self.edges[src, dst]['weight']
#         self.add_edge(src, dst, weight=weight)

In [4]:
cn = CitationNetwork(usc)
print(len(cn.nodes))

64175


In [24]:
title_sinks = {}

for i, sec_id in enumerate(cn.nodes):
    if i % 1000 == 0:
        print(i)
        
    title_id = util.extract_title_id(sec_id)
    node_sinks = cn.sinks_from(sec_id)
    title_sinks.setdefault(title_id, Counter()).update(node_sinks)
    
with open('../data/title_sinks_weighted.json') as f:
    json.dump(title_sinks, f)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


KeyboardInterrupt: 

In [5]:
with open('../data/title_sinks.json') as f:
    title_sinks = json.load(f)
    
for title in title_sinks:
    title_sinks[title] = set(title_sinks[title])

In [20]:
def distance(t1, t2):
    n_shared = len(title_sinks[t1.id] & title_sinks[t2.id])
    n_total = len(title_sinks[t1.id]) + len(title_sinks[t2.id]) - n_shared
    return 1 - n_shared / max(1, n_total)

titles = list(usc.titles())
clustering = Clustering(titles, distance)

In [21]:
clusters = clustering.get_clusters(5)
for c in clusters:
    print(c)

['t51', 't45', 't44', 't50', 't46', 't52', 't47', 't43', 't54', 't40', 't41', 't32', 't26', 't27', 't33', 't19', 't25', 't31', 't30', 't24', 't18', 't20', 't34', 't8', 't9', 't35', 't21', 't37', 't23', 't22', 't36', 't13', 't7', 't6', 't12', 't38', 't4', 't11', 't5', 't39', 't1', 't15', 't29', 't28', 't2', 't3', 't17', 't49', 't48']
['t42']
['t10']
['t14']
['t16']


In [ ]:
plt.figure(figsize=(5,5))
clustering.plot_dendrogram()